参考

- [Extracting Speech from Video using Python](https://towardsdatascience.com/extracting-speech-from-video-using-python-f0ec7e312d38)
  使用google的相关api，不能处理大的文件
 
- [SILERO SPEECH-TO-TEXT MODELS](https://pytorch.org/hub/snakers4_silero-models_stt/)
  采用pytorch中的torchaudio来实现

In [1]:
# for SpeechRecognition
#!pip install SpeechRecognition moviepy

# for torchaudio
#!pip install -q torchaudio omegaconf soundfile
# !apt-get update
#!apt-get install -y libsndfile1-dev

In [2]:
# ebablbe auto-completion
%config Completer.use_jedi = False

In [3]:
import speech_recognition as sr 
import moviepy.editor as mp
import os 

import torch
import zipfile
import torchaudio
from glob import glob

### Video to Audio 

支持的格式有

Video:
- MP4 (mp4, m4a, m4v, f4v, f4a, m4b, m4r, f4b, mov)
- 3GP (3gp, 3gp2, 3g2, 3gpp, 3gpp2)
- OGG (ogg, oga, ogv, ogx)
- WMV (wmv, wma, asf*)

Audio：
- MP3
- AAC
- WMA
- AC3 (Dolby Digital)

keyi

In [27]:
# video_file = "output/GMT20210920-213435_Recording.m4a"
def get_wav_text_path(audio_video_file):
    dirname = os.path.dirname(audio_video_file)
    basenameme = os.path.basename(audio_video_file)
    extension = os.path.splitext(audio_video_file)[1][1:]
    
    if extension=='wav':
        wav_file = audio_video_file
        text_file = os.path.join(dirname, basenameme + '.txt')
    else:
        wav_file = os.path.join(dirname, basenameme + '.wav')
        text_file = os.path.join(dirname, basenameme + '.txt')
        
    print(f'wav_file={wav_file}')
    print(f'text_file={text_file}')     
    return wav_file, text_file        

def to_wav(audio_video_file, wav_file=None):
    if wav_file is None:
        wav_file, _ = get_wav_text_path(audio_video_file)
    
    extension = os.path.splitext(audio_video_file)[1][1:]
    if wav_file==audio_video_file:
        return
    else:
        if extension in ['mp4']:
            clip = mp.VideoFileClip(audio_video_file) 
            clip.audio.write_audiofile(wav_file)
        else:
            clip = mp.AudioFileClip(audio_video_file) 
            clip.write_audiofile(wav_file)  
    return wav_file


def audio_to_text(model, wav_file, text_file):
    test_files = glob(wav_file)
    batches = split_into_batches(test_files, batch_size=10)
    input = prepare_model_input(read_batch(batches[0]),
                                device=device)

    output = model(input)
    for example in output:
        text = decoder(example.cpu())
        with open(text_file, mode ='w') as file: 
            file.write(text)     
    return text, output[0]
    

In [21]:
audio_video_file = "output/GMT20210920-213435_Recording_1920x1080.mp4"
wav_file = to_wav(audio_video_file)

wav_file=output/GMT20210920-213435_Recording_1920x1080.mp4.wav
text_file=output/GMT20210920-213435_Recording_1920x1080.mp4.txt


chunk:   1%|          | 162/27307 [00:00<00:16, 1616.59it/s, now=None]

MoviePy - Writing audio in output/GMT20210920-213435_Recording_1920x1080.mp4.wav


MoviePy - Done.


### 使用SpeechRecognition

In [14]:
audio_file = 'output/speech_orig.wav'

r = sr.Recognizer()
audio = sr.AudioFile(audio_file)

In [16]:
with audio as source:
    audio_file = r.record(source)
    
## 由于google的限制，大的文件按无法执行
result = r.recognize_google(audio_file)

In [17]:
# exporting the result 
with open(text_file, mode ='w') as file: 
    file.write(result) 

### 使用 torchaudio 

示例文件
~~~
# download a single file, any format compatible with TorchAudio (soundfile backend)
torch.hub.download_url_to_file('https://opus-codec.org/static/examples/samples/speech_orig.wav',
                               dst ='output/speech_orig.wav', progress=True)
~~~

对于大文件，如果采用cuda，会有如下错误。
~~~
RuntimeError: CUDA out of memory. Tried to allocate 7.14 GiB (GPU 0; 7.93 GiB total capacity; 429.57 MiB already allocated; 6.40 GiB free; 728.00 MiB reserved in total by PyTorch)
~~~

In [24]:
device = torch.device('cpu')  # gpu also works, but our models are fast enough for CPU

model, decoder, utils = torch.hub.load(repo_or_dir='snakers4/silero-models', 
                                       model='silero_stt',
                                       language='en', # also available 'de', 'es'
                                       device=device)
(read_batch, split_into_batches,
 read_audio, prepare_model_input) = utils  # see function signature for details
 


Using cache found in /root/.cache/torch/hub/snakers4_silero-models_master


In [29]:
text, output = audio_to_text(model, wav_file='output/speech_orig.wav', text_file='output/speech_orig.wav.txt')
print(text, output) 
print(type(output))

the boch canoeslid on the smooth planks blew the sheet to the dark blue background it's easy to tell a deps of a well four hours of steady work faced us tensor([[9.9945e-01, 1.4921e-07, 4.9026e-04,  ..., 6.5015e-06, 5.7053e-25,
         2.9590e-05],
        [1.6816e-04, 3.2007e-06, 9.9957e-01,  ..., 2.1278e-07, 2.8513e-18,
         9.9202e-06],
        [1.4031e-03, 4.3873e-07, 4.1116e-05,  ..., 1.0005e-04, 3.4989e-21,
         9.9840e-01],
        ...,
        [9.9926e-01, 1.5548e-06, 1.3937e-07,  ..., 1.0264e-06, 9.3222e-19,
         2.3636e-06],
        [9.9914e-01, 7.3186e-06, 1.4838e-05,  ..., 1.4995e-05, 3.0723e-14,
         2.3155e-04],
        [9.9914e-01, 7.3466e-06, 1.4920e-05,  ..., 1.4955e-05, 3.1072e-14,
         2.3055e-04]])
<class 'torch.Tensor'>


In [ ]:
tyoe(output

In [25]:
audio_video_file = "output/GMT20210920-213435_Recording_1920x1080.mp4"
wav_file, text_file = get_wav_text_path(audio_video_file)

wav_file=output/GMT20210920-213435_Recording_1920x1080.mp4.wav
text_file=output/GMT20210920-213435_Recording_1920x1080.mp4.txt


In [22]:
to_wav(audio_video_file, wav_file)

chunk:   1%|          | 163/27307 [00:00<00:16, 1628.81it/s, now=None]

MoviePy - Writing audio in output/GMT20210920-213435_Recording_1920x1080.mp4.wav


MoviePy - Done.


'output/GMT20210920-213435_Recording_1920x1080.mp4.wav'

In [26]:
text = audio_to_text(model, wav_file, text_file)
print(text)

we going show you help work the so we have to go from we have to lect d p then we have to lect this is the we have to to look for the re d which is the bres cl then we have to one is from the engineers which is the one the the right s and is and the the one for the part request the fl and so first we going to ready t request from the vie the engineer we have exam or ready lect that re way to for the d so mentation before we to t request ity the day this or the par or the the serv to the custo so this case custo is request to change the dis the of the iceso going show to request from so from the thisis the for we will go aheadand select the workor so we will go aheadand the will click on the work order numberand lect the work order clo the from here we will change the request from yes to noand click say then we have to w the the the request rely so we need to vie on the work or book the will that be and together which is right and casual re be customer request so both spes we the right 